In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
from itertools import chain

plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_Jul14_ntuple/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

In [4]:
hists = {}

hists["hist_pdgid1"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"PDG ID", 40, -400, 4000)
                            )
hists["hist_pdgid2"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"PDG ID", 40, -400, 4000)
                            )
hists["hist_ke1"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"Kinetic Energy [MeV]", 50, 4, 3500),
                                )
hists["hist_ke2"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"Kinetic Energy [MeV]", 50, 4, 3500),
                                )

In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)
    
    sim_particle = ldmx_events['Sim_PNParticle']
    sim_particle_n = ldmx_events['n']['Sim_PNParticle']
    
    masks_id = {'Kplus': 321,
                'Kshort': 310,
                'Klong': 130, 
                'Kminus':-321,
                'Electron': 11,
                'Positron': -11,
                'Pion': 211,
                'Pi-': -211,
                'Proton':2212,
                'Neutron':2112,
                'Sigma': 3212,
                'Lambda': 3122,
               }
    masses =   {'Kplus': 493.677,
                'Kshort': 493.677,
                'Klong': 493.677,
                'Kminus': 493.677,
                'Electron': 0.511,
                'Positron': 0.511,
                'Pion': 139.57039,
                'Pi-': 139.57039,
                'Proton': 939.565,
                'Neutron': 939.565,
                'Sigma': 1189.36,
                'Lambda': 1115.6,
                }

    sim_particle_ke = sim_particle.e
    print(sim_particle_ke)
    for part,pmask in masks_id.items():
        part_mask = sim_particle.pdgID == pmask
        print(sim_particle.e[part_mask] - masses[part])
        sim_particle_ke[part_mask] = sim_particle.e[part_mask] - masses[part]
        
    print(sim_particle_ke)
    print(sim_particle.e)
    
    index = ak.argsort(sim_particle.e, axis=-1, ascending=False, stable=True, highlevel=True, behavior=None)
    index = ak.pad_none(index, 2, axis=1)

    padded_pdg = ak.pad_none(sim_particle.pdgID, 2, axis=1)
    pdgID_1 = padded_pdg[index][:,0]
    pdgID_2 = padded_pdg[index][:,1]
    filled_pdg1 = ak.fill_none(pdgID_1, -1000000)
    filled_pdg2 = ak.fill_none(pdgID_2, -1000000)
    
    padded_e = ak.pad_none(sim_particle.e, 2, axis=1)
    e_1 = padded_e[index][:,0]
    e_2 = padded_e[index][:,1]
    filled_e1 = ak.fill_none(e_1, -1000000)
    filled_e2 = ak.fill_none(e_2, -1000000)

    kshort_mask = filled_pdg1 == 310
    klong_mask  = filled_pdg1 == 130
    kplus_mask  = filled_pdg1 == 321
    kminus_mask = filled_pdg1 == -321
    kaon_mask = kshort_mask | klong_mask | kplus_mask | kminus_mask
    n_kaons = ak.sum(kaon_mask)
    n_kaon_mask = n_kaons == 1
    
    sigma_mask = filled_pdg1 == 3212
    k_anymask = ak.any(kshort_mask)
    sigma_anymask = ak.any(sigma_mask)
    
    total_mask = k_anymask & sigma_anymask & n_kaon_mask
   # print(filled_pdg1)
    #print(filled_pdg1[k_anymask])
    #print(filled_pdg1[total_mask])
    
    for part,pmask in masks_id.items():
        part_mask1 = (filled_pdg1 == pmask)
        hists["hist_pdgid1"].fill(Ptype= part,
                                     e=flat(filled_pdg1[total_mask][part_mask1]),
                                     )  
        hists["hist_ke1"].fill(Ptype= part,
                                     e=flat(filled_e1[total_mask][part_mask1])- masses[part],
                                     )   
#         part_mask2 = (filled_pdg2[total_mask] == pmask)
#         hists["hist_pdgid2"].fill(Ptype= part,
#                                      e=flat(filled_pdg2[total_mask][part_mask2]),
#                                      )  
#         hists["hist_ke2"].fill(Ptype= part,
#                                      e=flat(filled_e2[total_mask][part_mask2])- masses[part],
#                                      )   
    
    #goal: plot for events with 1 Kshort (no other kaons) and 1 sigma

In [6]:
nchunk = 0

for chunk in ldmx_dict:
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)


process 1
[[], [1.02], [180, 741, 425, ... 210, 1.43e+03, 1.69e+03, 1.08e+03], [13.4, 17.6]]
[[], [], [], [], [], [], [], [], [], [], ... [], [], [], [398], [], [], [], [], []]


TypeError: only fields may be assigned in-place (by field name)

(https://github.com/scikit-hep/awkward-1.0/blob/1.2.3/src/awkward/highlevel.py#L1055)

In [ ]:
#48 kminus
print(hists["hist_pdgid1"])
fig, ax = plt.subplots(1,2, figsize=(24,10), constrained_layout=True)
hist.plot1d(hists["hist_pdgid1"]+hists["hist_pdgid2"],ax=ax[0],clear=False);
#leg = ax[0].legend([r'$K_S$', r'$\Lambda$', r'$n^0$', r'$\pi^-$', r'$\pi^+$',r'$p^+$', r'$\Sigma$'])


hist.plot1d(hists["hist_ke1"]+hists["hist_ke2"],ax=ax[1],clear=False);
#leg = ax[1].legend([r'$K_S$', r'$\Lambda$', r'$n^0$', r'$\pi^-$', r'$\pi^+$',r'$p^+$', r'$\Sigma$'])

